In [1]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)


for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")
print('train x ' + str(train_x))
print('train y ' + str(train_y))


# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=100, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ndhar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ndhar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Using TensorFlow backend.


348 documents
8 classes ['age', 'begin', 'country', 'gender', 'severity', 'start', 'stop', 'symptom']
392 unique lemmatized words ['&', ',', '.', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '5', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '6', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '7', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '8', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '9', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', 'afghanistan', 'africa', 'african', 'albania', 'algeria', 'american', 'and', 'andorra', 'angola', 'anguilla', 'antigua', 'antilles', 'arab', 'arabia', 'argentina', 'armenia', 'aruba', 'australia', 'austria', 'azerbaijan', 'bahamas', 'bahrain', 'bangladesh', 'bank', 'ba

Epoch 1/100
348/348 [==============================] - 1s 3ms/step - loss: 1.6964 - accuracy: 0.5690
Epoch 2/100
348/348 [==============================] - 0s 657us/step - loss: 0.9959 - accuracy: 0.6523
Epoch 3/100
348/348 [==============================] - 0s 807us/step - loss: 0.8690 - accuracy: 0.6609
Epoch 4/100
348/348 [==============================] - 0s 723us/step - loss: 0.8335 - accuracy: 0.6523
Epoch 5/100
348/348 [==============================] - 0s 713us/step - loss: 0.7738 - accuracy: 0.7098
Epoch 6/100
348/348 [==============================] - 0s 692us/step - loss: 0.7261 - accuracy: 0.6868
Epoch 7/100
348/348 [==============================] - 0s 670us/step - loss: 0.6358 - accuracy: 0.7586
Epoch 8/100
348/348 [==============================] - 0s 643us/step - loss: 0.5065 - accuracy: 0.8534
Epoch 9/100
348/348 [==============================] - 0s 695us/step - loss: 0.4160 - accuracy: 0.8908
Epoch 10/100
348/348 [==============================] - 0s 608us/step - los

348/348 [==============================] - 0s 527us/step - loss: 0.0345 - accuracy: 0.9828
Epoch 80/100
348/348 [==============================] - 0s 632us/step - loss: 0.0335 - accuracy: 0.9885
Epoch 81/100
348/348 [==============================] - 0s 647us/step - loss: 0.0374 - accuracy: 0.9770
Epoch 82/100
348/348 [==============================] - 0s 633us/step - loss: 0.0282 - accuracy: 0.9885
Epoch 83/100
348/348 [==============================] - 0s 602us/step - loss: 0.0312 - accuracy: 0.9885
Epoch 84/100
348/348 [==============================] - 0s 604us/step - loss: 0.0311 - accuracy: 0.9856
Epoch 85/100
348/348 [==============================] - 0s 593us/step - loss: 0.0315 - accuracy: 0.9885
Epoch 86/100
348/348 [==============================] - 0s 598us/step - loss: 0.0401 - accuracy: 0.9828
Epoch 87/100
348/348 [==============================] - 0s 597us/step - loss: 0.0361 - accuracy: 0.9856
Epoch 88/100
348/348 [==============================] - 0s 590us/step - loss: